In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from config import *

In [2]:
record_time = np.zeros(int(T/PERIOD))
for t in range(1, T+1):
    if t % PERIOD == 0:
        record_time[int(t/PERIOD)-1] = t

In [3]:
def record(old_array, record_time, to_be_record, baseline, n, dataset, distribution, prob):
    for t,v in zip(record_time, to_be_record):
        if old_array is None:
            old_array = np.array([baseline, n, dataset, distribution, prob, t, v])
        else:
            old_array = np.vstack((old_array, np.array([baseline, n, dataset, distribution, prob, t, v])))
    return old_array

In [4]:
all_data = None
for BASELINE in BASELINES:
    for N in Ns:
        for DATASET in DATASETS:
            for DISTRIBUTION in DISTRIBUTIONS:
                if BASELINE == "MOTS":
                    if DISTRIBUTION in ["Gamma", "Poisson"]:
                        continue
                if BASELINE in ["TSGreedy", "ExpTS_plus"]:
                    for PROB in PROBS_fn(N):
                        data = np.load("./results/{}-{}-{}-{}-{}.npy".format(BASELINE, N, DATASET, DISTRIBUTION, round(PROB, 2)))
                        all_data = record(all_data, record_time, data, BASELINE, N, DATASET, DISTRIBUTION, PROB)
                else:
                    data = np.load("./results/{}-{}-{}-{}-{}.npy".format(BASELINE, N, DATASET, DISTRIBUTION, 1.0))
                    all_data = record(all_data, record_time, data, BASELINE, N, DATASET, DISTRIBUTION, 1.0)

In [ ]:
import pandas as pd
eval_path = "./results/results.xlsx"
col = np.array(["baseline", "n", "dataset", "distribution", "prob", "t", "eval"])
df = pd.DataFrame(data, columns=col)
df.to_excel(eval_path)